In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import dotenv
os.chdir("/home/sdonoso/projects/GPTChat")
dotenv.load_dotenv()
! pwd

/home/sdonoso/projects/GPTChat


In [4]:
from google.cloud import storage


def download_blob_into_memory(bucket_name, blob_name):
    """Downloads a blob into memory."""
    # The ID of your GCS bucket
    bucket_name = "gpt-las-chat"

    # The ID of your GCS object
    blob_name = "gpt-las-chat/data_chat"

    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)

    # Construct a client side representation of a blob.
    # Note `Bucket.blob` differs from `Bucket.get_blob` as it doesn't retrieve
    # any content from Google Cloud Storage. As we don't need additional data,
    # using `Bucket.blob` is preferred here.
    blob = bucket.blob(blob_name)
    contents = blob.download_as_bytes()

    print(
        "Downloaded storage object {} from bucket {} as the following bytes object: {}.".format(
            blob_name, bucket_name, contents.decode("utf-8")
        )
    )


In [5]:
def download_bucket_with_transfer_manager(
    bucket_name, destination_directory="", workers=8, max_results=1000
):
    """Download all of the blobs in a bucket, concurrently in a process pool.

    The filename of each blob once downloaded is derived from the blob name and
    the `destination_directory `parameter. For complete control of the filename
    of each blob, use transfer_manager.download_many() instead.

    Directories will be created automatically as needed, for instance to
    accommodate blob names that include slashes.
    """

    # The ID of your GCS bucket
    # bucket_name = bucket_name = "gpt-las-chat"

    # The directory on your computer to which to download all of the files. This
    # string is prepended (with os.path.join()) to the name of each blob to form
    # the full path. Relative paths and absolute paths are both accepted. An
    # empty string means "the current working directory". Note that this
    # parameter allows accepts directory traversal ("../" etc.) and is not
    # intended for unsanitized end user input.
    # destination_directory = ""

    # The maximum number of processes to use for the operation. The performance
    # impact of this value depends on the use case, but smaller files usually
    # benefit from a higher number of processes. Each additional process occupies
    # some CPU and memory resources until finished. Threads can be used instead
    # of processes by passing `worker_type=transfer_manager.THREAD`.
    # workers=8

    # The maximum number of results to fetch from bucket.list_blobs(). This
    # sample code fetches all of the blobs up to max_results and queues them all
    # for download at once. Though they will still be executed in batches up to
    # the processes limit, queueing them all at once can be taxing on system
    # memory if buckets are very large. Adjust max_results as needed for your
    # system environment, or set it to None if you are sure the bucket is not
    # too large to hold in memory easily.
    # max_results=1000

    from google.cloud.storage import Client, transfer_manager

    storage_client = Client()
    bucket = storage_client.bucket(bucket_name)

    blob_names = [blob.name for blob in bucket.list_blobs(prefix="data_chat",max_results=max_results)]

    results = transfer_manager.download_many_to_path(
        bucket, blob_names, destination_directory=destination_directory, max_workers=workers
    )

    for name, result in zip(blob_names, results):
        # The results list is either `None` or an exception for each blob in
        # the input list, in order.

        if isinstance(result, Exception):
            print("Failed to download {} due to exception: {}".format(name, result))
        else:
            print("Downloaded {} to {}.".format(name, destination_directory + name))

In [6]:
from datetime import datetime

while True:
    print("Current time:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    time.sleep(60)
    download_bucket_with_transfer_manager(bucket_name="gpt-las-chat",destination_directory="/home/sdonoso/projects/GPTChat/stats/data")

Current time: 2024-09-26 23:08:51
Failed to download data_chat/ due to exception: [Errno 21] Is a directory: '/home/sdonoso/projects/GPTChat/stats/data/data_chat/'
Downloaded data_chat/uploaded_data_20240913174524.json to /home/sdonoso/projects/GPTChat/stats/datadata_chat/uploaded_data_20240913174524.json.
Downloaded data_chat/uploaded_data_20240913174525.json to /home/sdonoso/projects/GPTChat/stats/datadata_chat/uploaded_data_20240913174525.json.
Downloaded data_chat/uploaded_data_20240913174624.json to /home/sdonoso/projects/GPTChat/stats/datadata_chat/uploaded_data_20240913174624.json.
Downloaded data_chat/uploaded_data_20240913174625.json to /home/sdonoso/projects/GPTChat/stats/datadata_chat/uploaded_data_20240913174625.json.
Downloaded data_chat/uploaded_data_20240913174656.json to /home/sdonoso/projects/GPTChat/stats/datadata_chat/uploaded_data_20240913174656.json.
Downloaded data_chat/uploaded_data_20240913182513.json to /home/sdonoso/projects/GPTChat/stats/datadata_chat/uploade

In [19]:
from stats.count_votes import  count_votes
votes = count_votes("/home/sdonoso/projects/GPTChat/stats/data/data_chat/*.json")

0it [00:00, ?it/s]


In [20]:
votes

Counter()